In [197]:
import numpy as np
from math import *
from scipy.linalg import expm, fractional_matrix_power

#########################################################################

# pauli matrix 
sx = np.array([[0,  1],     [1, 0]])
sy = np.array([[0, -1j],   [1j, 0]])
sz = np.array([[1, 0],     [0, -1]])
s0 = np.array([[1, 0],      [0, 1]])

# parameters(detuning factor)
v0 = 0.02           # Arbitrary settings, Actual speed : 0.04rad/μs
d0 = 0.15           # Arbitrary settings, Actual speed : 0.30rad/μs
# dt = 2.6 


# unitary operator
def unitary(dt, choice):
    
    # Select x,y-rotation direction.
    # [stay, +x, -x, +y, -y]
    choice_list = [0, 1, -1, 1, -1] 
    
    if choice < 3:
        # if choice = 0 ... only d0*sz
        Ham = (d0*sz+v0*choice_list[choice]*sx)
    else:
        Ham = (d0*sz+v0*choice_list[choice]*sy)

    # Creating a Unitary Operator for each of the four sections by Hamiltonian
    eigvals = np.linalg.eigh(Ham)[0]
    eigvecs = 1*np.linalg.eigh(Ham)[1]
    E = np.diag(eigvals)
    U_H = eigvecs.conj().T
    U_e = U_H.conj().T @ expm(-1j*E*dt) @ U_H
    
    return U_e


#########################################################################

# x-rotation operater
def Rx(theta):
    return np.matrix([  [cos(theta/2),    -1j*sin(theta/2)],
                        [-1j*sin(theta/2),    cos(theta/2)]])

# z-rotation operater
# Do not use Rz. Control by rotation only by Hamiltonian.
def Rz(phi): 
    return np.matrix([  [cos(phi/2)-1j*sin(phi/2),  0],
                        [0,  cos(phi/2)+1j*sin(phi/2)]])

# Calculating the Fidelity
def state_fidelity(rho_1, rho_2): 
    
    # rho_1(current state), rho_2(target state)
    # Calculate the fidelity after checking the dimensions of the two states.
    
    if np.shape(rho_1) != np.shape(rho_2):
        print("Dimensions of two states do not match.")
        return 0
    else:
        sqrt_rho_1 = fractional_matrix_power(rho_1, 1 / 2)
        fidelity = np.trace(fractional_matrix_power(sqrt_rho_1 @ rho_2 @ sqrt_rho_1, 1 / 2)) ** 2
        return np.real(fidelity)


#########################################################################

values = np.arange(1.820, 4.001, 0.001)
# 반올림하여 3자리 소수점까지만 표시
dt_values = [round(val, 3) for val in values]

unitary_choiceList = {}

for dt in dt_values:
    unitary_choiceList[dt] = [unitary(dt, i) for i in range(5)]


init_wave = np.array([[1], [0]])
irho_init = np.kron(init_wave,init_wave.conj().T)


def fidelity(rho, dt, predicted_seq):

    # predicted_seq = np.argmax(predicted_seq, axis=-1)
    # if type(predicted_seq) != list:
    #     predicted_seq = np.argmax(predicted_seq, axis=-1).astype(int).tolist()
    
    Uni = s0
    
    for i in predicted_seq:
        Uni = unitary_choiceList[dt][i] @ Uni

    irho_final = Uni @ irho_init @ Uni.conj().T
    
    F = (state_fidelity(irho_final, rho))
    
    return  F

In [198]:
# def rho_making(dt, predicted_comb):
     
#     # rho = np.matrix(irho_init)
#     rho = irho_init
    
#     for i in predicted_comb:
#         instant_U = unitary_choiceList[dt][i]
#         rho = (instant_U @ rho @ instant_U.conj().T)
     
#     return rho


def find_best_dt(seq):
     
    global input_rho
     
    my_dt = 0
    fid = 0         # final fidelity
    i_fid = 0       # temp fidelity
    
    for i in dt_values:
        
        # irho_new = rho_making(i, seq)
        # i_fid = state_fidelity(irho_new, input_rho)
        i_fid = fidelity(input_rho, i, seq)
    
        if fid < i_fid:
            my_dt = i
            fid = i_fid
    
    return my_dt

In [199]:
from itertools import product

def find_best_seq(seq):
     
     global input_rho
     
     final_seq = []
     temp_seq = []
     temp_fid = 0
     temp_dt = 2.6
     
     # for i in range(-1, -(len(seq)+1), -1):
     for i in range(-1, -8, -1):
          
          if i < -len(seq):
               break
          
          front_seq = seq[:i]
          back_seq = seq[i:]
          
          for combination in product(range(5), repeat=len(back_seq)):
               temp_seq = front_seq + list(combination)
               
               fid = fidelity(input_rho, temp_dt, temp_seq)     # default dt=2.6
               
               if fid > 0.99990:
                    final_seq = temp_seq
                    return temp_dt, final_seq
               elif fid > temp_fid:
                    final_seq = temp_seq
                    temp_fid = fid

               # print("=============================")
               # print(seq)
               # print(temp_seq)
               # print(final_seq)
               # print("=============================")

     if temp_fid < 0.99990:
          final_dt = find_best_dt(final_seq)
     else:
          final_dt = temp_dt
     
     return final_dt, final_seq

In [200]:
import time

def local_optimizer(input_rho, input_seq):

     first_fid = fidelity(input_rho, 2.6, input_seq)        # default dt=2.6

     if first_fid > 0.99990:
          output_seq = input_seq
     else:
          start_time = time.time()
          
          output_dt, output_seq = find_best_seq(input_seq)
          
          end_time = time.time()
          computing_t = end_time - start_time

     last_fid = fidelity(input_rho, output_dt, output_seq)

     total_t = output_dt * len(output_seq)
     
     return output_dt, output_seq, first_fid, last_fid, total_t, computing_t

In [201]:
import os
from datetime import datetime
import pandas as pd

# Create a directory to store the results
dir = 'LO_results_dir'
if not os.path.exists(dir):
    os.makedirs(dir)

# CSV file name setup
date = datetime.now()
printdate = date.strftime('%Y%m%d_%H%M%S')
# filename = "/ByLO_dt_" + str(dt) + "_" + printdate + '.csv'
filename = "/ByLO_dt_" + "change_" + printdate + '.csv'

# Create an empty DataFrame and write to CSV file
saved_df = pd.DataFrame(columns=["Case", 'gate length', 'Theta', 'Phi', 'dt', 'combination', 'fidelity',
                           'total time', 'computing time'])
saved_df.to_csv(dir + filename, index=False)

In [202]:
import pandas as pd

df = pd.read_csv('../Data/using/dt_2.6/ByAstar_dt_2.6_1016.csv')
df = df[df['combination'] != '[]']
df = df.drop('Case', axis=1)
df_shuffled = df.sample(frac=1).reset_index(drop=True)

thetas = df_shuffled['Theta']
phis = df_shuffled['Phi']
lists = df_shuffled['combination']


df_shuffled.head()

,gate length,Theta,Phi,dt,combination,total time,computing time
0,32,3.100289,0.582969,2.6,"[1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, ...",83.2,36.743589
1,26,2.134124,4.144520,2.6,"[2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, ...",67.6,400.488163
2,18,1.271543,6.039484,2.6,"[4, 4, 1, 1, 3, 3, 2, 4, 1, 1, 1, 1, 3, 2, 2, ...",46.8,1927.698780
3,33,2.737157,4.991284,2.6,"[2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, ...",85.8,872.642022
4,4,0.537675,5.799751,2.6,"[2, 4, 4, 4]",10.4,12.729845


In [203]:
case = 0

for input_theta, input_phi, input_sequence in zip(thetas, phis, lists):  

     target_U = Rz(input_phi) @ Rx(input_theta)
     input_rho = target_U @ irho_init @ target_U.conj().T
     
     output_dt, output_sequence, first_fidelity, last_fidelity, total_time, computing_time = local_optimizer(input_rho, eval(input_sequence))

     case += 1
     output = [['case' + str(case), len(output_sequence), input_theta, input_phi, output_dt, output_sequence, 
                    last_fidelity, total_time, computing_time]]
    
     # Create DataFrame and append to CSV file
     saved_df = pd.DataFrame(output, columns=["Case", 'gate length', 'Theta', 'Phi', 'dt', 'combination', 
                                                  'fidelity', 'total time', 'computing time'])
     saved_df.to_csv(dir + filename, mode='a', header=False, index=False)

     # Result output
     print(f"""
     ------------------------------------------------------------------------------------------------------
     theta = {input_theta}
     phi = {input_phi}
     selected_dt : {output_dt}
     input_sequence : {input_sequence}
     first_fidelity : {first_fidelity}
     output_sequence : {output_sequence}
     last_fidelity : {last_fidelity}
     total_time : {total_time}
     computing_time : {computing_time}
     ------------------------------------------------------------------------------------------------------
     """)
     
     print(case, "/", len(df_shuffled))



     ------------------------------------------------------------------------------------------------------
     theta = 3.100289489
     phi = 0.582968772
     selected_dt : 2.619
     input_sequence : [1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 2, 4]
     first_fidelity : 0.9910687150580402
     output_sequence : [1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4]
     last_fidelity : 0.9965832492446753
     total_time : 83.808
     computing_time : 213.84992694854736
     ------------------------------------------------------------------------------------------------------
     
1 / 27806

     ------------------------------------------------------------------------------------------------------
     theta = 2.134124023
     phi = 4.144520196
     selected_dt : 2.606
     input_sequence : [2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 1, 1, 3, 4]
     first_fidelity : 0.9